In [1]:
import requests # for scraping
import pandas as pd # for data manipulation / handling
from datetime import datetime # for formatting dates
import logging

# Data given to me by Eric
# Getting only docket ids from that file
FILENAME = '19-20.csv' # CHANGE THIS DAD
docket_ids = pd.read_csv(FILENAME)['Docket ID'].drop_duplicates().tolist()
# logging.basicConfig(filename='docket_scraping.log', level=logging.INFO)

# Key under "Tristan Cooper"
# HAD TO APPLY FOR THIS
api_key = 'QQZvi5FMGwQPT1XuFWT5tDzvs4obqSbbhe8XmhH6' 

def fetch_docket_title(docket_id, api_key):
    docket_url = f"https://api.regulations.gov/v4/dockets/{docket_id}"
    headers = {"X-Api-Key": api_key}
    docket_response = requests.get(docket_url, headers=headers)
    
    if docket_response.status_code == 200:
        docket_data = docket_response.json()
        return docket_data['data']['attributes']['title']
    else:
        print(f"Error fetching title for docket ID {docket_id}: {docket_response.status_code}, Details: {docket_response.text}")
        return None

def fetch_commenters_info(docket_id, api_key):
    url = f"https://api.regulations.gov/v4/comments?filter[docketId]={docket_id}"
    headers = {"X-Api-Key": api_key}
    response = requests.get(url, headers=headers)
    commenters_info = []
    
    docket_title = fetch_docket_title(docket_id, api_key)
    if not docket_title:
        return []
    
    if response.status_code == 200:
        data = response.json()
        
        for comment in data['data']:
            title = comment['attributes']['title']
            date_iso = comment['attributes']['postedDate']
            date = date_iso.split('T')[0]
            submitted_by_prefix = "Comment Submitted by "
            if title.startswith(submitted_by_prefix) and "Anonymous" not in title:
                org_name = title[len(submitted_by_prefix):]
                commenters_info.append({
                    'Docket ID': docket_id,
                    'Docket Name': docket_title,
                    'Commenter Name': org_name,
                    'Date': date
                })
        return commenters_info
    else:
        error_details = response.text
        print(f"Oh no tristan couldn't fetch data for: {docket_id}: {response.status_code}, Details: {error_details}")
        return []

all_commenters_info = []

# Loop through all docket IDs 
# for docket_id in docket_ids:    
#     all_commenters_info.extend(fetch_commenters_info(docket_id, api_key))

for docket_id in docket_ids:
    # logging.info(f"Processing docket ID: {docket_id}")
    try:
        commenters_info = fetch_commenters_info(docket_id, api_key)
        if commenters_info:
            all_commenters_info.extend(commenters_info)
            logging.info(f"Successfully processed docket ID: {docket_id}")
        else:
            logging.warning(f"No data returned for docket ID: {docket_id}")
    except Exception as e:
        logging.error(f"Error processing docket ID {docket_id}: {e}")

all_commenters_df = pd.DataFrame(all_commenters_info)



/var/folders/ld/s8p1wyns2txf6czmb7pkx43r0000gn/T/ipykernel_4564/119647723.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd # for data manipulation / handling


Error fetching title for docket ID USCG-2019-0721: 429, Details: {
  "error": {
    "code": "OVER_RATE_LIMIT",
    "message": "You have exceeded your rate limit. Try again later or contact us at https://www.regulations.gov/support for assistance"
  }
}
Error fetching title for docket ID USCG-2019-0722: 429, Details: {
  "error": {
    "code": "OVER_RATE_LIMIT",
    "message": "You have exceeded your rate limit. Try again later or contact us at https://www.regulations.gov/support for assistance"
  }
}
Error fetching title for docket ID USCG-2019-0725: 429, Details: {
  "error": {
    "code": "OVER_RATE_LIMIT",
    "message": "You have exceeded your rate limit. Try again later or contact us at https://www.regulations.gov/support for assistance"
  }
}
Error fetching title for docket ID USCG-2019-0727: 429, Details: {
  "error": {
    "code": "OVER_RATE_LIMIT",
    "message": "You have exceeded your rate limit. Try again later or contact us at https://www.regulations.gov/support for assist

In [2]:
all_commenters_df['Total Comments'] = all_commenters_df.groupby('Commenter Name')['Commenter Name'].transform('count')
# all_commenters_df.tail()
# print rows that have more than 1 comment
# all_commenters_df[all_commenters_df['Total Comments'] > 1]

# print number of unique docket ids in the date fram
# all_commenters_df['Docket ID'].nunique()
# len(docket_ids)


73

In [1]:
# all_commenters_df.shape
# ex = pd.read_csv('regulations.csv')

# ex['Docket ID'].nunique()

# len(all_commenters_df['Docket ID'].unique())

# len(docket_ids)
# print(all_commenters_df.shape)

all_commenters_df.to_csv('{FILENAME}_commenters.csv', index=False)
# all_commenters_df.head()

NameError: name 'all_commenters_df' is not defined

In [84]:
all_commenters_df["Total Comments"].value_counts

<bound method IndexOpsMixin.value_counts of 0      1
1      1
2      1
3      1
4      1
      ..
190    1
191    2
192    1
193    2
194    1
Name: Total Comments, Length: 195, dtype: int64>